In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf

import numpy as np

print(tf.__version__)

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64


(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

2.3.0


In [3]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 75s 120ms/step - loss: 0.4225 - acc: 0.8046 - val_loss: 0.3470 - val_acc: 0.8662
Epoch 2/10
625/625 [==============================] - 74s 119ms/step - loss: 0.2390 - acc: 0.9094 - val_loss: 0.2922 - val_acc: 0.8848
Epoch 3/10
625/625 [==============================] - 74s 118ms/step - loss: 0.1836 - acc: 0.9319 - val_loss: 0.2865 - val_acc: 0.8968
Epoch 4/10
625/625 [==============================] - 76s 121ms/step - loss: 0.1509 - acc: 0.9459 - val_loss: 0.2873 - val_acc: 0.8884
Epoch 5/10
625/625 [==============================] - 75s 120ms/step - loss: 0.1293 - acc: 0.9547 - val_loss: 0.3123 - val_acc: 0.8848
Epoch 6/10
625/625 [==============================] - 74s 118ms/step - loss: 0.1145 - acc: 0.9618 - val_loss: 0.2965 - val_acc: 0.8860
Epoch 7/10
625/625 [==============================] - 72s 115ms/step - loss: 0.0972 - acc: 0.9690 - val_loss: 0.3546 - val_acc: 0.8852
Epoch 8/10
625/625 [==============================] - 7

In [4]:
results = model.evaluate(test_data, test_labels)
print(results)

#%
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

782/782 [==============================] - 24s 31ms/step - loss: 0.4800 - acc: 0.8567
[0.4799555838108063, 0.856719970703125]
1646592/1641221 [==============================] - 1s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0

In [5]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [6]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.93359685]
[0.38363642]


In [ ]:
# Play generation
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
print(path_to_file)

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))
print(text[:250])

In [ ]:
# encode
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])



text_as_int = text_to_int(text)

# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))
print(int_to_text(text_as_int[:13]))

In [ ]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
#Next we can use the batch method to turn this stream of characters into batches 
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
print(sequences)

In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)
print(dataset)

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))

In [ ]:
# create training batches
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

In [ ]:
# Loss function
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

In [ ]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

In [9]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

C:\Users\Andreas\.keras\datasets\shakespeare.txt


In [10]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))
print(text[:250])

Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [12]:
# encode
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])



text_as_int = text_to_int(text)

# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))
print(int_to_text(text_as_int[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]
First Citizen


In [13]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [15]:
#Next we can use the batch method to turn this stream of characters into batches 
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
print(sequences)

<BatchDataset shapes: (101,), types: tf.int32>


In [17]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)
print(dataset)

<MapDataset shapes: ((100,), (100,)), types: (tf.int32, tf.int32)>


In [18]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [20]:
# create training batches
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [21]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Loss function
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [23]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-8.10732599e-03  1.28454459e-03  6.43549312e-04 ... -1.23100297e-04
   -3.76847852e-03 -1.33665814e-03]
  [-5.37005719e-03 -2.67456076e-03 -3.30269802e-03 ... -2.34984327e-03
    2.04892666e-03  4.46558977e-03]
  [-1.40979595e-03 -3.19663761e-03 -6.41988683e-03 ... -1.25414954e-04
   -2.25309399e-04  5.39946230e-03]
  ...
  [-1.41652511e-03 -1.33200949e-02 -1.32215675e-03 ...  1.53856049e-03
   -1.32704363e-03  3.11582605e-03]
  [-3.90638132e-03 -1.21561661e-02  9.86884348e-04 ...  3.90194100e-03
    2.18167296e-03  1.63995149e-03]
  [-1.74674043e-03 -1.35085462e-02 -3.13620013e-03 ...  1.13303214e-03
    5.60330879e-03  7.56025035e-03]]

 [[ 2.90910690e-03 -2.12667207e-03  9.55655612e-03 ... -5.07801655e-04
   -1.04479189e-03  6.88892137e-03]
  [ 3.46407248e-03 -7.30794203e-03  2.98870448e-03 ... -2.92648072e-03
    5.84115647e-03  8.90666246e-03]
  [ 8.74965824e-03 -1.59736408e-03 -3.61318281e-03 ... -3.26658343e-03
    8.94947909e-03  7.70928455e-04]
  ...
  [ 1.259

In [24]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00810733  0.00128454  0.00064355 ... -0.0001231  -0.00376848
  -0.00133666]
 [-0.00537006 -0.00267456 -0.0033027  ... -0.00234984  0.00204893
   0.00446559]
 [-0.0014098  -0.00319664 -0.00641989 ... -0.00012541 -0.00022531
   0.00539946]
 ...
 [-0.00141653 -0.01332009 -0.00132216 ...  0.00153856 -0.00132704
   0.00311583]
 [-0.00390638 -0.01215617  0.00098688 ...  0.00390194  0.00218167
   0.00163995]
 [-0.00174674 -0.01350855 -0.0031362  ...  0.00113303  0.00560331
   0.00756025]], shape=(100, 65), dtype=float32)


In [25]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-8.1073260e-03  1.2845446e-03  6.4354931e-04  3.9793514e-03
 -2.8175437e-03 -3.1024881e-03 -1.6372800e-03  4.7764834e-04
  3.3297979e-03 -3.4942443e-04 -2.2513201e-03  1.6056071e-04
  1.8885987e-03 -4.0045548e-03 -6.5391522e-04 -4.1217529e-03
 -4.2110342e-03 -4.4678617e-03  3.2384628e-03  2.3067431e-03
  5.6732334e-03 -8.6032040e-04 -1.3159506e-03 -3.9123208e-03
  2.1334905e-03 -3.5581626e-03  2.4828082e-03  1.2455152e-03
 -2.0108838e-04 -1.8162437e-03 -1.6570918e-03  2.8189551e-03
  1.1046596e-03  5.2460814e-03  6.3025663e-03 -8.3947438e-05
  6.5005547e-04  2.2041542e-03 -3.0050101e-03 -5.6278007e-04
  1.4905022e-03  5.3270918e-04 -1.3152547e-03 -4.2042714e-03
 -6.3619977e-03 -5.5820053e-03  9.0459367e-04  3.2273808e-04
  2.8965631e-03 -1.1569010e-03  9.4950451e-03  2.9212437e-03
  2.6008049e-03 -2.0516026e-03 -7.8662415e-04  6.1713043e-05
  1.3328278e-03 -4.6540946e-03 -2.7024797e-03  3.6181430e-03
  2.7597288e-03  1.3742589e-03 -1.2310030e-04 -3.7684785e-03
 -1.336658

In [26]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'Gfks;PfTAq.Our?Bk!?dBwruzEnz:tes$gi,D 3c$TWs,aMYCxQtMODFsBdQU,,,\nBWuqw:fkoxe$r.vzBJwHtMOh&LkGR\nYk? d'

In [27]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [28]:
model.compile(optimizer='adam', loss=loss)

In [29]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 1173s 7s/step - loss: 2.5545
Epoch 2/50
125/172 [====================>.........] - ETA: 12:07 - loss: 1.9270